In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow import keras


from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


from keras.layers import SimpleRNN, Embedding, Dense, LSTM, Dropout, SimpleRNN, CuDNNLSTM
from keras.models import Sequential






Using TensorFlow backend.


In [1]:
from google.colab import files
uploaded = files.upload()


Saving emo.csv to emo.csv


In [4]:
import io
inputs = pd.read_csv(io.StringIO(uploaded['emo.csv'].decode('utf-8')))
inputs.head()
print(len(inputs))

1600


In [5]:
inputs = inputs.dropna()
print(len(inputs))
print(inputs)

1600
       Decision     anger      fear  ...   disgust  positive  negative
0      truthful  0.009524  0.009524  ...  0.000000  0.066667  0.028571
1      truthful  0.000000  0.000000  ...  0.000000  0.111111  0.000000
2      truthful  0.004831  0.014493  ...  0.004831  0.091787  0.033816
3      truthful  0.007874  0.015748  ...  0.015748  0.055118  0.031496
4      truthful  0.000000  0.000000  ...  0.000000  0.055556  0.000000
...         ...       ...       ...  ...       ...       ...       ...
1595  deceptive  0.007634  0.007634  ...  0.000000  0.076336  0.038168
1596  deceptive  0.008264  0.000000  ...  0.008264  0.066116  0.000000
1597  deceptive  0.007092  0.007092  ...  0.014184  0.085106  0.014184
1598  deceptive  0.034884  0.011628  ...  0.046512  0.069767  0.046512
1599  deceptive  0.015385  0.007692  ...  0.019231  0.061538  0.023077

[1600 rows x 11 columns]


In [6]:
df2 = inputs.reindex(np.random.permutation(inputs.index))
print(df2)

       Decision     anger      fear  ...   disgust  positive  negative
1015   truthful  0.011765  0.017647  ...  0.017647  0.000000  0.029412
318    truthful  0.004545  0.009091  ...  0.000000  0.100000  0.018182
1277  deceptive  0.014925  0.014925  ...  0.022388  0.044776  0.029851
845    truthful  0.011811  0.011811  ...  0.007874  0.035433  0.043307
89     truthful  0.000000  0.000000  ...  0.000000  0.085470  0.000000
...         ...       ...       ...  ...       ...       ...       ...
1534  deceptive  0.000000  0.008000  ...  0.004000  0.040000  0.028000
1353  deceptive  0.013333  0.006667  ...  0.006667  0.060000  0.013333
1478  deceptive  0.011583  0.015444  ...  0.030888  0.030888  0.042471
658   deceptive  0.000000  0.000000  ...  0.000000  0.151515  0.000000
867    truthful  0.013699  0.013699  ...  0.013699  0.031963  0.018265

[1600 rows x 11 columns]


In [7]:
labels = df2['Decision'].values.tolist()
print(labels)

['truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'truthful', 'truthful', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'truthful', 'deceptive', 'deceptive', 'deceptive', 'truthful', 'truthful', 'truthful', 'truthful', 'truthful', 'truthful', 'truthful', 'deceptive', 'truthful', 'tr

In [0]:
labels = np.array([0 if label == "deceptive" else 1 for label in labels])

In [9]:
print(labels)
print(len(labels))

[1 1 0 ... 0 0 1]
1600


In [10]:
del df2['Decision']
print(df2)

         anger      fear  anticipation  ...   disgust  positive  negative
1015  0.011765  0.017647      0.011765  ...  0.017647  0.000000  0.029412
318   0.004545  0.009091      0.054545  ...  0.000000  0.100000  0.018182
1277  0.014925  0.014925      0.007463  ...  0.022388  0.044776  0.029851
845   0.011811  0.011811      0.019685  ...  0.007874  0.035433  0.043307
89    0.000000  0.000000      0.034188  ...  0.000000  0.085470  0.000000
...        ...       ...           ...  ...       ...       ...       ...
1534  0.000000  0.008000      0.032000  ...  0.004000  0.040000  0.028000
1353  0.013333  0.006667      0.013333  ...  0.006667  0.060000  0.013333
1478  0.011583  0.015444      0.027027  ...  0.030888  0.030888  0.042471
658   0.000000  0.000000      0.000000  ...  0.000000  0.151515  0.000000
867   0.013699  0.013699      0.018265  ...  0.013699  0.031963  0.018265

[1600 rows x 10 columns]


In [11]:
train_x = df2.to_numpy()
print(len(train_x))
a = train_x[0]
print(a)

1600
[0.01176471 0.01764706 0.01176471 0.         0.01176471 0.00588235
 0.         0.01764706 0.         0.02941176]


In [0]:
split_frac1 = 0.8

idx1 = int(len(train_x) * split_frac1)
train_x, val_x = train_x[:idx1], train_x[idx1:]
train_y, val_y = labels[:idx1], labels[idx1:]

In [13]:
#print(train_x)
print(len(train_x))
print(len(train_y))
print(len(val_x))
print(len(val_y))


1280
1280
320
320


In [0]:
clf = LogisticRegression()
clf.fit(train_x,train_y)
p = clf.predict(val_x)
print (accuracy_score(train_y,p))

Basic NN
 

In [17]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(rate=0.25))

model.add(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid))


model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))



model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs= 50, callbacks=[TestCallback((val_x,val_y))])



    # model.fit(train_x,train_y,epochs=,batch_size=64,verbose=1)
    # _, accuracy = model.evaluate(val_x, val_y)
    # print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/50
30/40 [=====================>........] - ETA: 0s - loss: 0.6932 - accuracy: 0.5167
Testing loss: 0.6922942996025085, acc: 0.5406249761581421

40/40 [==============================] - 0s 8ms/step - loss: 0.6933 - accuracy: 0.5102 - val_loss: 0.6923 - val_accuracy: 0.5406
Epoch 2/50
29/40 [====================>.........] - ETA: 0s - loss: 0.6933 - accuracy: 0.5129
Testing loss: 0.6927060484886169, acc: 0.4625000059604645

40/40 [==============================] - 0s 6ms/step - loss: 0.6930 - accuracy: 0.5133 - val_loss: 0.6927 - val_accuracy: 0.4625
Epoch 3/50
30/40 [=====================>........] - ETA: 0s - loss: 0.6913 - accuracy: 0.5458
Testing loss: 0.6890760064125061, acc: 0.5718749761581421

40/40 [==============================] - 0s 6ms/step - loss: 0.6922 - accuracy: 0.5281 - val_loss: 0.6891 - val_accuracy: 0.5719
Epoch 4/50
28/40 [====================>.........] - ETA: 0s - loss: 0.6883 - accuracy: 0.5502
Testing loss: 0.6849371790885925, acc: 0.578125

40/40 [====

In [16]:

_, accuracy = model.evaluate(val_x, val_y)
print('Accuracy: %.2f' % (accuracy*100))

10/10 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.4625
Accuracy: 46.25
